In [ ]:
# Getting necessary packages
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import psycopg2
from psycopg2 import sql 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 

In [44]:
pwd=os.getcwd()
filepath=os.path.join(pwd,"ageing_data.csv")
filepath

'c:\\Users\\Ernest\\Desktop\\Python\\Ageing_data\\ageing_data.csv'

In [45]:
ageing_data = pd.read_csv(filepath,low_memory=False)
ageing_data.head() 

,RowId,YearStart,YearEnd,LocationAbbr,LocationDesc,Class,Topic,Question,Response,DataValueTypeID,...,StratificationCategory2,Stratification2,Geolocation,ClassID,TopicID,QuestionID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2
0,BRFSS~2015~2015~9003~Q43~TOC11~AGE~OVERALL,2015,2015,SOU,South,Overall Health,Arthritis among older adults,Percentage of older adults ever told they have...,NaN,PRCTG,...,NaN,NaN,NaN,C01,TOC11,Q43,AGE,5064,OVERALL,OVERALL
1,BRFSS~2021~2021~01~Q03~TMC01~AGE~GENDER,2021,2021,AL,Alabama,Mental Health,Frequent mental distress,Percentage of older adults who are experiencin...,NaN,PRCTG,...,Gender,Female,POINT (-86.63186076199969 32.84057112200048),C05,TMC01,Q03,AGE,AGE_OVERALL,GENDER,FEMALE
2,BRFSS~2021~2021~41~Q27~TMC03~AGE~RACE,2021,2021,OR,Oregon,Mental Health,Lifetime diagnosis of depression,Percentage of older adults with a lifetime dia...,NaN,PRCTG,...,Race/Ethnicity,Hispanic,POINT (-120.15503132599969 44.56744942400047),C05,TMC03,Q27,AGE,AGE_OVERALL,RACE,HIS
3,BRFSS~2021~2021~31~Q27~TMC03~AGE~OVERALL,2021,2021,NE,Nebraska,Mental Health,Lifetime diagnosis of depression,Percentage of older adults with a lifetime dia...,NaN,PRCTG,...,NaN,NaN,POINT (-99.36572062299967 41.6410409880005),C05,TMC03,Q27,AGE,AGE_OVERALL,OVERALL,OVERALL
4,BRFSS~2021~2021~18~Q27~TMC03~AGE~GENDER,2021,2021,IN,Indiana,Mental Health,Lifetime diagnosis of depression,Percentage of older adults with a lifetime dia...,NaN,PRCTG,...,Gender,Female,POINT (-86.14996019399968 39.766910452000445),C05,TMC03,Q27,AGE,AGE_OVERALL,GENDER,FEMALE


In [46]:
ageing_data.shape

(250937, 25)

In [47]:
ageing_data.columns.to_list() 

['RowId',
 'YearStart',
 'YearEnd',
 'LocationAbbr',
 'LocationDesc',
 'Class',
 'Topic',
 'Question',
 'Response',
 'DataValueTypeID',
 'Data_Value_Type',
 'Data_Value',
 'Data_Value_Alt',
 'StratificationCategory1',
 'Stratification1',
 'StratificationCategory2',
 'Stratification2',
 'Geolocation',
 'ClassID',
 'TopicID',
 'QuestionID',
 'StratificationCategoryID1',
 'StratificationID1',
 'StratificationCategoryID2',
 'StratificationID2']

In [48]:
password = input("Please enter your Password: ") 

In [49]:
#Establishing a connection
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="psycopg_final",
    user="postgres",
    password=password)
conn

<connection object at 0x000001D158A71AD0; dsn: 'user=postgres password=xxx dbname=psycopg_final host=localhost port=5432', closed: 0>

In [50]:
# Set the isolation level to autocommit
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [51]:
# Create a cursor object
cursor = conn.cursor() 

In [52]:
# Define the table name
table_name = 'Elderly_Registry' 

In [53]:
# Define the CREATE TABLE statement
create_table_query = sql.SQL('''
    CREATE TABLE IF NOT EXISTS {} (
        "RowId" VARCHAR(255),
        "YearStart" INTEGER,
        "YearEnd" INTEGER,
        "LocationAbbr" VARCHAR(255),
        "LocationDesc" VARCHAR(255),
        "Class" VARCHAR(255),
        "Topic" VARCHAR(255),
        "Question" VARCHAR(255),
        "Response" VARCHAR(255),
        "DataValueTypeID" VARCHAR(255),
        "Data_Value_Type" VARCHAR(255),
        "Data_Value" FLOAT,
        "Data_Value_Alt" FLOAT,
        "StratificationCategory1" VARCHAR(255),
        "Stratification1" VARCHAR(255),
        "StratificationCategory2" VARCHAR(255),
        "Stratification2" VARCHAR(255),
        "Geolocation" VARCHAR(255),
        "ClassID" VARCHAR(10),
        "TopicID" VARCHAR(10),
        "QuestionID" VARCHAR(25),
        "StratificationCategoryID1" VARCHAR(255),
        "StratificationID1" VARCHAR(255),
        "StratificationCategoryID2" VARCHAR(255),
        "StratificationID2" VARCHAR(255)
    )
''').format(sql.Identifier(table_name))

In [54]:
#Execute the CREATE TABLE statement
cursor.execute(create_table_query) 

In [55]:
# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Ernest\\Desktop\Python\\Ageing_data\\ageing_data.csv'

# Define the COPY command to load data from CSV
#copy_command = sql.SQL('COPY {} FROM %s WITH (FORMAT CSV, DELIMITER "," , HEADER)').format(
 #   sql.Identifier(table_name)
#)
# Define the COPY command to load data from CSV
copy_command = sql.SQL('COPY {} FROM STDIN WITH (FORMAT CSV, DELIMITER ",", HEADER)').format(
    sql.Identifier(table_name)
)
# Execute the COPY command with the CSV file path
with open(csv_file_path, 'r') as f:
    cursor.copy_expert(copy_command, f)

# Commit the changes
conn.commit()

In [56]:
# Close the cursor and the database connection
cursor.close()
conn.close()